<a href="https://colab.research.google.com/github/vivi1231/0814/blob/master/lymphoma_generator_tf_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
from glob import glob
import cv2
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.models import Model

In [6]:
IMG_SIZE = 200
BATCH_SIZE = 2
class_map = {'CLL':0, 'MCL': 1, 'FL': 2}

In [7]:
# Read single image
data_path = 'drive/My Drive/Day4/lymphoma/*/*.tif' 
img_paths = glob(data_path)
img_paths_train, img_paths_test = train_test_split(img_paths, test_size=0.2)
len(img_paths), len(img_paths_train), len(img_paths_test)

(374, 299, 75)

In [8]:
# yield用法似return，但他可以不用跳出迴圈，可以直接等下一批資料進來
# image_data = x_train  class_data = y_train
def data_generator(data_paths, batch_size, num_classes):
    '''data generator for fit_generator'''
    n = len(data_paths)
    i = 0
    data_paths = data_paths
    while True:
        image_data = []
        class_data = []
        for b in range(batch_size):
            if i==0:
                np.random.shuffle(data_paths)
            path = data_paths[i]
            img = cv2.imread(path)
            img_resize = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            img_resize = img_resize/255
            cls = path.split('/')[-2]

            image_data.append(img_resize)
            class_data.append(class_map[cls])
            i = (i+1) % n
        image_data = np.array(image_data)
        class_data = np.array(class_data)
        class_data = tf.keras.utils.to_categorical(class_data, num_classes=num_classes)
        yield image_data, class_data


In [9]:
generator_train = data_generator(data_paths=img_paths_train, batch_size=BATCH_SIZE, num_classes=3)
generator_test = data_generator(data_paths=img_paths_test, batch_size=BATCH_SIZE, num_classes=3)

In [10]:
# 1  建模
inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = Conv2D(filters=16, kernel_size=3, activation='relu')(inputs)
x = Conv2D(filters=16, kernel_size=3, activation='relu')(x)
x = MaxPool2D(2)(x)
x = Conv2D(filters=32, kernel_size=3, activation='relu')(x)
x = Conv2D(filters=32, kernel_size=3, activation='relu')(x)
x = MaxPool2D(2)(x)
x = Flatten()(x)
x = Dense(64, activation=tf.keras.activations.relu)(x)
prediction = Dense(3, activation='softmax')(x)
model = Model(inputs=inputs, outputs=prediction)

In [11]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 200, 200, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 198, 198, 16)      448       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 196, 196, 16)      2320      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 98, 98, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 96, 96, 32)        4640      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 94, 94, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 47, 47, 32)       

In [12]:
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [13]:
# generator_train = (x_train, y_tarin)

logs = model.fit(generator_train,
                epochs=10,
                steps_per_epoch=len(img_paths_train)//BATCH_SIZE,
                validation_data=generator_test,
                validation_steps=len(img_paths_test)//BATCH_SIZE,
                )

Epoch 1/10
149/149 [==============================] - 653s 4s/step - loss: 1.1327 - accuracy: 0.3356 - val_loss: 1.1036 - val_accuracy: 0.2568
Epoch 2/10
149/149 [==============================] - 9s 58ms/step - loss: 1.0959 - accuracy: 0.3993 - val_loss: 1.1048 - val_accuracy: 0.2703
Epoch 3/10
149/149 [==============================] - 9s 58ms/step - loss: 1.0937 - accuracy: 0.4027 - val_loss: 1.1107 - val_accuracy: 0.2703
Epoch 4/10
149/149 [==============================] - 9s 57ms/step - loss: 1.1275 - accuracy: 0.3893 - val_loss: 1.1078 - val_accuracy: 0.2838
Epoch 5/10
149/149 [==============================] - 9s 58ms/step - loss: 1.0930 - accuracy: 0.3960 - val_loss: 1.1174 - val_accuracy: 0.2568
Epoch 6/10
149/149 [==============================] - 8s 57ms/step - loss: 1.0910 - accuracy: 0.4027 - val_loss: 1.1233 - val_accuracy: 0.2703
Epoch 7/10
149/149 [==============================] - 9s 57ms/step - loss: 1.0897 - accuracy: 0.4027 - val_loss: 1.1221 - val_accuracy: 0.2703

In [ ]:
history = logs.history

In [ ]:
plt.plot(history['accuracy'])
plt.plot(history['val_accuracy'])
plt.legend(['accuracy', 'val_accuracy'])
plt.title('accuracy')

In [ ]:
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.title('loss')

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
y_true = np.argmax(y_test, axis=-1)
y_pred = np.argmax(model.predict(X_test), axis=-1)
print(y_true.shape, y_pred.shape)
target_names = [str(i) for i in range(3)]
print(classification_report(y_true, y_pred, target_names=target_names))
print(confusion_matrix(y_true, y_pred))
